In [1]:
import pandas as pd
import spacy

from gensim.models import LdaModel

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

import nltk
import re
from gensim.models import Phrases
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
from gensim.corpora import Dictionary
nltk.download('omw-1.4')

#import pyLDAvis.gensim_models
import warnings


#from snowballstemmer import stemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# ! pip install spacy

### Lecture des fichiers

In [3]:
df = pd.read_csv(r'C:\Users\arceb\PycharmProjects\Projet_DSSS_2023/fullcorpus20210725_DATE.csv', encoding = 'ISO-8859-1').drop('Unnamed: 0',axis=1)
df.replace({
        '\xa0':'',
        '\x80':'',
        'Ã\x89':'É',
        'â\x99':' ',
        'Ã§':'ç',
        'Ã©': 'é', 
        'Ã¨': 'è', 
        'Ãª': 'ê', 
        'Ã´': 'ô',
        'Ã': 'à',
        'Â': '', 
        'â': "'", 
        'à ': 'à',
    }, regex=True, inplace = True)

print(df.shape)
df.head()

(8026, 9)


,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"Éric Albert, Cécile Boutelet, Jérôme Gautheret...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,Le Figaro (site web),Élargissement de lâUnion européenne à lâheur...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique française ...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement dâun ...,Figaro,migrant_migratoire
3,Le Monde (site web),Coronavirus: les trois semaines qui ont chambo...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26mars, les Vingt-Sept doivent se réunir...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, nâest pas ...",Monde,migrant_migratoire
4,Le Monde (site web),Marc Lazar: «Plus lâItalie fait nation face à...,2020-03-25,Marc Lazar,Tribune. LâItalie est le pays qui enregistre ...,date du 25mars le plus grand nombre de morts ...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire


### Préparation des données

In [4]:
# Testing
df = df.loc[df['Journal'].str.contains('Monde', regex=True)].head(10)
df.shape

(10, 9)

In [5]:
# Import du package
import spacy

!pip install numpy=='1.22.1'
!pip install --upgrade h5py # ça m'a debloqué en redémarrant le kernel après ça

# Installation de la pipline entrainée pour la langue française
!python -m spacy download fr_core_news_sm

ERROR: Could not find a version that satisfies the requirement numpy=='1.22.1' (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.21.2, 1.21.3, 1.21.4, 1.21.5, 1.21.6, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0rc1, 1.23.0rc2, 1.23.0rc3, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23.4, 1.23.5, 1.24.0rc1, 1.24.0rc2, 1.24.0, 1.24.1, 1.24.2, 1.24.3, 1.24.4, 1.25.0rc1, 1.25.0, 1.25.1, 1.25.2, 1.26.0b1, 1.26.0rc1, 1.26.0, 1.26.1, 1.26.2, 1.26.3, 1.26.4)
ERROR: No mat

In [11]:
nlp = spacy.load("fr_core_news_sm")

OSError: [E050] Can't find model 'fr_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
textes = df["Texte"]

In [ ]:
%%time

spacy_docs = list(nlp.pipe(textes))

Prétraitement  
1) supprimer tous les mots de moins de 3 caractères,  # debatable  
2) supprimer tous les stop-words, et  
3) lemmatiser les mots restants et,  
4) mettre ces mots en minuscules.

In [ ]:
%%time

docs = []
for doc in spacy_docs:
    tokens = []
    for token in doc:
        if len(token.orth_) > 3 and not token.is_stop: # prétraitements 1 et 2
            tokens.append( token.lemma_.lower() )  # TODO: prétraitements 3 et 4
    docs.append( tokens )

Afin de conserver un peu l'ordre des mots lors de notre modélisation, nous allons tenir en compte les bigrammes fréquents. Pour cela, nous allons utiliser la bibliothèque Gensim (excellente bibliothèque NLP pour les topics modeling et word embeddings)

Voici le processus retenu :

- Identifier les bigrammes fréquents dans le corpus,  
- Ajouter à la liste des tokens pour les documents dans lesquels ils apparaissent.  

Cela signifie que les bigrammes ne seront pas dans leur position correcte dans le texte, mais comme les topic models sont des modèles de bag-of-words (sac-de-mots en français) qui ignorent la position des mots, cela ne pose pas de problèmes

In [ ]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for index in range(len(docs)):
    for token in bigram[docs[index]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[index].append(token) # TODO

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary()

# Ajouter les documents un par un au dictionnaire
for doc in docs:
    dictionary.add_documents([doc])
    
print('Nombre de mots unique dans les documents initiaux :', len(dictionary))

# dictionary.filter_extremes(no_below=3, no_above=0.25)
# print('Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents :', len(dictionary))

print("Exemple :", dictionary.doc2bow(docs[4]))

In [ ]:
corpus = [ dictionary.doc2bow(doc) for doc in docs] # TODO

In [ ]:
%%time
from gensim.models import LdaModel

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1) # A REMPLIR

In [ ]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

Visualisation

In [ ]:
import pyLDAvis.gensim_models
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)

In [ ]:
display(pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False))
a=5

In [ ]:
# Nous en affichons que 4
n_doc = 4
i = 0
for (text, doc) in zip(textes[:n_doc], docs[:n_doc]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

# Boucle par journaux X années

In [ ]:
%%time
nlp = spacy.load("fr_core_news_sm")
for year in range(2009, 2021):
    for journal in df.CJournal.unique().tolist():
        data = df.loc[(df['Date'].str.contains(str(year), regex=True))&(df['CJournal']==journal), ["Titre"]]
        
        textes = data["Titre"]
        try :
            spacy_docs = list(nlp.pipe(textes))

            docs = []
            for doc in spacy_docs:
                tokens = []
                for token in doc:
                    if len(token.orth_) > 3 and not token.is_stop: # prétraitements 1 et 2
                        tokens.append( token.lemma_.lower() )  # TODO: prétraitements 3 et 4
                docs.append( tokens )

            bigram = Phrases(docs, min_count=10)

            for index in range(len(docs)):
                for token in bigram[docs[index]]:
                    if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
                        docs[index].append(token) # TODO

            dictionary = Dictionary()

            # Ajouter les documents un par un au dictionnaire
            for doc in docs:
                dictionary.add_documents([doc])

            corpus = [ dictionary.doc2bow(doc) for doc in docs]
            model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1) # A REMPLIR

            print(year, journal, len(data), " articles")
            pyLDAvis.enable_notebook()
            warnings.filterwarnings("ignore", category=DeprecationWarning) 

            display(pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False))
        except :
            continue

In [ ]:
# Nous en affichons que 4
n_doc = 4
i = 0
for (text, doc) in zip(textes[:n_doc], docs[:n_doc]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

# Boucle par journaux X années

In [ ]:
%%time
nlp = spacy.load("fr_core_news_sm")
for year in range(2009, 2021):
    for journal in df.CJournal.unique().tolist():
        data = df.loc[(df['Date'].str.contains(str(year), regex=True))&(df['CJournal']==journal), ["Titre"]]
        
        textes = data["Titre"]
        try :
            spacy_docs = list(nlp.pipe(textes))

            docs = []
            for doc in spacy_docs:
                tokens = []
                for token in doc:
                    if len(token.orth_) > 3 and not token.is_stop: # prétraitements 1 et 2
                        tokens.append( token.lemma_.lower() )  # TODO: prétraitements 3 et 4
                docs.append( tokens )

            bigram = Phrases(docs, min_count=10)

            for index in range(len(docs)):
                for token in bigram[docs[index]]:
                    if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
                        docs[index].append(token) # TODO

            dictionary = Dictionary()

            # Ajouter les documents un par un au dictionnaire
            for doc in docs:
                dictionary.add_documents([doc])

            corpus = [ dictionary.doc2bow(doc) for doc in docs]
            model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1) # A REMPLIR

            print(year, journal, len(data), " articles")
            pyLDAvis.enable_notebook()
            warnings.filterwarnings("ignore", category=DeprecationWarning) 

            display(pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False))
        except :
            continue

2009 Monde 1  articles


C:\Users\tlp\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=         x    y  topics  cluster       Freq
topic                                      
0      0.0 -0.0       1        1   0.833335
1      0.0 -0.0       2        1   0.833335
2     -0.0  0.0       3        1   0.833335
3     -0.0 -0.0       4        1   0.833335
4     -0.0 -0.0       5        1  92.499983
5     -0.0 -0.0       6        1   0.833335
6     -0.0  0.0       7        1   0.833335
7      0.0  0.0       8        1   0.833335
8      0.0  0.0       9        1   0.833335
9     -0.0  0.0      10        1   0.833335, topic_info=        Term      Freq  Total Category  logprob  loglift
1      calai  1.000000    1.0  Default  11.0000     11.0
7     jungle  1.000000    1.0  Default  10.0000     10.0
6       fort  0.000000    0.0  Default   9.0000      9.0
3   collègue  0.000000    0.0  Default   8.0000      8.0
0     besson  0.000000    0.0  Default   7.0000      7.0
..       ...       ...    ...      ...      ...      ...
1      calai  0.008333    1.0  Topic10  -2.3979     -0.0
0     besson  0.008333    1.0  Topic10  -2.3979     -0.0
6       fort  0.008333    1.0  Topic10  -2.3979     -0.0
3   collègue  0.008333    1.0  Topic10  -2.3979     -0.0
7     jungle  0.008333    1.0  Topic10  -2.3979     -0.0

[121 rows x 6 columns], token_table=      Topic  Freq      Term
term                       
0         5   1.0    besson
1         5   1.0     calai
2         5   1.0    caméra
3         5   1.0  collègue
4         5   1.0   envoyer
5         5   1.0      eric
6         5   1.0      fort
7         5   1.0    jungle
8         5   1.0    signal
9         5   1.0   vouloir
10        5   1.0   évacuer, R=11, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

2009 Croix 2  articles


C:\Users\tlp\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


...

C:\Users\tlp\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.083709  0.089757       1        1  17.486566
1      0.015813 -0.015602       2        1   8.332827
2      0.091484  0.060023       3        1  12.943274
3     -0.012544  0.083865       4        1   9.120498
4     -0.139116  0.058652       5        1  10.555172
5      0.060722 -0.060301       6        1   7.169275
6      0.063852 -0.068118       7        1   9.738208
7     -0.068737  0.006101       8        1   7.589779
8     -0.088520 -0.047097       9        1   5.491616
9     -0.006663 -0.107280      10        1  11.572783, topic_info=               Term       Freq      Total Category  logprob  loglift
0           migrant  51.000000  51.000000  Default  30.0000  30.0000
333       frontière   4.000000   4.000000  Default  29.0000  29.0000
80            video   9.000000   9.000000  Default  28.0000  28.0000
148         diviser   3.000000   3.000000  Default  27.0000  27.0000
330        contrôle   4.000000   4.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
225          sortie   0.739550   2.028522  Topic10  -5.1327   1.1475
199          france   0.739550   3.847338  Topic10  -5.1327   0.5074
336  hauts-de-seine   0.739550   1.913545  Topic10  -5.1327   1.2058
46         boulogne   0.739548   1.940346  Topic10  -5.1327   1.1919
261            aide   0.739546   2.710248  Topic10  -5.1327   0.8578

[505 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
360       5  0.788622      2014
424       3  0.515376      2015
424       4  0.515376      2015
26        4  0.410415      2016
26        9  0.410415      2016
...     ...       ...       ...
107       6  0.336482     édito
107       8  0.336482     édito
107       9  0.336482     édito
25        9  0.843147  élection
393      10  0.765886     élire

[624 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

2015 Echos 153  articles


C:\Users\tlp\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.115202 -0.060679       1        1  16.652723
1     -0.043550 -0.059224       2        1  11.136787
2     -0.029311  0.004870       3        1   2.991312
3     -0.125250 -0.103775       4        1  13.434127
4      0.025449 -0.016089       5        1   9.657214
5      0.109682  0.035518       6        1  13.334876
6      0.030467  0.050269       7        1   7.497282
7     -0.052080  0.027659       8        1   4.004117
8      0.040142 -0.030918       9        1   9.678673
9     -0.070751  0.152369      10        1  11.612887, topic_info=              Term       Freq      Total Category  logprob  loglift
38          merkel  16.000000  16.000000  Default  30.0000  30.0000
0           europe  19.000000  19.000000  Default  29.0000  29.0000
42         réfugié  24.000000  24.000000  Default  28.0000  28.0000
15   international   3.000000   3.000000  Default  27.0000  27.0000
105       hollande   6.000000   6.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
19           renzi   0.732265   2.411150  Topic10  -4.9786   0.9613
437         raison   0.732265   1.981223  Topic10  -4.9786   1.1577
461      demandeur   0.732265   1.858302  Topic10  -4.9786   1.2218
88            pays   0.732265   3.346889  Topic10  -4.9786   0.6334
353     volkswagen   0.732265   1.900391  Topic10  -4.9786   1.1994

[492 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
444       5  0.809913    10.000
421       7  0.838497   120.000
392       7  0.838497      2016
393       7  0.838497    85.000
135       4  0.773246   aborder
...     ...       ...       ...
468       1  0.760150  égoà¯sme
458       4  0.773247   élargir
493       7  0.838496   épreuve
305       4  0.773248    étaler
309       4  0.773248    éviter

[522 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])